<h1>Creating a 3D Print</h1>
In this notebook we will use everything we've learned to create an STL file to send to our 3D printer.<br>
The methodology works, but unfortunately the model we created was not able to be printed in it's form.

---

<h2>Prerequisites</h2>
<ol>
<li><a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/1.%20Setting%20Up%20the%20API%20Connection.ipynb'>Setting Up the API Connection</a></li>
<li><a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/2.%20Using%20the%20Spotify%20API.ipynb'>Using the Spotify API</a></li>
<li><a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/3.%20Understanding%20the%20Data.ipynb'>Understanding the Data</a></li>
<li><a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/4.%20Creating%20a%203D%20Music%20Model.ipynb'>Creating a 3D Music Model</a></li>
</ol>

---

<h2>Create an STL File</h3>

<h3>Import Libraries</h3>

In [1]:
import math
import spotipy
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from spotipy.oauth2 import SpotifyClientCredentials

<h3>Create Functions</h3>
We will condense everything we learned in our first 4 notebooks into three, convenient functions.

<h4>Connect to Spotify API</h4>
From <a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/1.%20Setting%20Up%20the%20API%20Connection.ipynb'>Setting Up the API Connection</a>.

In [2]:
def connect_to_spotify():
    sp_client_id = input("Enter your Spotify Client ID: ")
    sp_client_secret = input("Enter your Spotify Secret Token: ")
    
    credentials = SpotifyClientCredentials(client_id = sp_client_id,
                                           client_secret = sp_client_secret)
    
    sp = spotipy.Spotify(client_credentials_manager = credentials)
    
    print("Connected to Spotify!")

    return sp

<h4>Get Track URI</h4>
From <a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/2.%20Using%20the%20Spotify%20API.ipynb'>Using the Spotify API</a>.

In [3]:
def get_uri(connection):

    print('\nRetrieving Track URI.')

    artist = input("Input artist name: ")
    track = input("Input track name: ")

    search_term = f'artist:{artist} track:{track}'

    result = connection.search(search_term, type = 'track')['tracks']['items'][0]

    # If no result, return to user
    if not result:
        return print("Search returned no result")
    else:
        print("\nTrack found!")

    track_uri = result['uri']

    artist_name = result['artists'][0]['name']

    track_name = result['name']

    print(f'\nTrack name: {track_name}')
    print(f'Artist name: {artist_name}')
    print(f'Track URI: {track_uri}')

    return track_uri

<h4>Create Timbre DataFrame</h4>
From <a href = 'https://nbviewer.org/github/JonYarber/music_modeling/blob/main/4.%20Creating%20a%203D%20Music%20Model.ipynb'>Creating a 3D Music Model</a>.

In [4]:
def create_timbre_df(uri):

   # Use the track URI to get the audio analysis. We only need the 'segments' portion
    segments = sp.audio_analysis(uri)['segments']

    # Convert it to a dataframe
    segment_df = pd.DataFrame(segments)

    # Create the timbre dataframe by selecting only the columns we need: start, loudness_start, timbre
    timbre_df = segment_df[['start', 'loudness_start', 'timbre']].copy()

    # Rename loudness and start columns
    timbre_df = timbre_df.rename(columns = {'loudness_start': 'loudness', 'start': 'start_time'})

    # Expand timbres
    for i in range(12):
        timbre_df[f'timbre_{i + 1}'] = timbre_df['timbre'].apply(lambda x: x[i])

    # Drop the original timbre column
    timbre_df.drop(['timbre'], axis = 1, inplace = True)

    # Add '13th timbre'
    timbre_df['timbre_13'] = timbre_df.timbre_1

    # Create a list of the timbre columns
    timbre_cols = [col for col in timbre_df.columns if col.startswith('timbre_')]
    
    # Melt the timbres
    timbre_df = timbre_df.melt(id_vars = ['start_time', 'loudness'],
                               value_vars = timbre_cols,
                               var_name = 'timbre_which',
                               value_name = 'timbre')
    
    # Create the 'timbre_num' column by removing the the word 'timbre' in the 'timbre_which' column
    timbre_df['timbre_num'] = [int(timbre_which.split('_')[1]) for timbre_which in timbre_df['timbre_which']]
    
    # Drop the timbre_which column
    timbre_df.drop(['timbre_which'], axis = 1, inplace = True)
    
    # Sort the dataframe by start_time and timbre_num
    timbre_df = timbre_df.sort_values(by = ['start_time', 'timbre_num']).reset_index(drop = True)

    # Apply the log transform
    timbre_df['timbre_log'] = np.log10(timbre_df['timbre'] + 400)

    # Apply the MinMax scaler to the timbre column
    timbre_df['timbre_scaled'] = MinMaxScaler().fit_transform(timbre_df['timbre'].values.reshape(-1, 1))

    # Adjust values to bounds [1, 2]
    timbre_df['timbre_scaled'] = timbre_df['timbre_scaled'] + 1

    # Apply MinMax scaler to the loudness column. 
    timbre_df['loudness_scaled'] = MinMaxScaler().fit_transform(timbre_df['loudness'].values.reshape(-1, 1))

    # Adjust values to bounds [1, 2]
    timbre_df['loudness_scaled'] = timbre_df['loudness_scaled'] + 1

    #  Scale timbre_log with loudness
    timbre_df['timbre_log_w_loud'] = timbre_df['loudness_scaled'] * timbre_df['timbre_log']
    
    # Scale timbre_scaled with loudness
    timbre_df['timbre_scaled_w_loud'] = timbre_df['loudness_scaled'] * timbre_df['timbre_scaled']

    # Create a column of angles based on timbre number (timbre_num)
    timbre_df['timbre_angle'] = 30 * timbre_df['timbre_num']
    
    # The sin and cos functions in the math package will only take the angles as radians. 
    # Convert the column just created to radians
    timbre_df['timbre_angle'] = (math.pi * timbre_df['timbre_angle']) / 180
    
    # Create timbre_X
    # For this, we want to use the timbre_log_w_loud variable
    timbre_df['timbre_X'] = round(timbre_df['timbre_log_w_loud'] * timbre_df['timbre_angle'].apply(lambda z: math.cos(z)), 3)
    
    # Create timbre_Y using timbre_log_w_loud
    timbre_df['timbre_Y'] = round(timbre_df['timbre_log_w_loud'] * timbre_df['timbre_angle'].apply(lambda z: math.sin(z)), 3)
    
    # We don't need the angle column anymore
    timbre_df.drop(['timbre_angle'], axis = 1, inplace = True)

    return timbre_df

<h3>Create the Timbre DataFrame</h3>
Now we use our functions to create our timbre dataframe.

In [5]:
# Don't run if already done
if 'sp' not in locals():
    sp = connect_to_spotify()

# Search and retrieve a track URI
track_uri = get_uri(sp)

# Create timbre dataframe
timbre_df = create_timbre_df(track_uri)

Enter your Spotify Client ID:  ed9307841d3542df8819aec9a4f0ec84
Enter your Spotify Secret Token:  8208717955574be6a24163ed59675094


Connected to Spotify!

Retrieving Track URI.


Input artist name:  Bon Jovi
Input track name:  Livin On a Prayer



Track found!

Track name: Livin' On A Prayer
Artist name: Bon Jovi
Track URI: spotify:track:37ZJ0p5Jm13JPevGcx4SkF


In [6]:
# Check 
timbre_df.head()

,start_time,loudness,timbre,timbre_num,timbre_log,timbre_scaled,loudness_scaled,timbre_log_w_loud,timbre_scaled_w_loud,timbre_X,timbre_Y
0,0.0,-60.0,0.171,1,2.602246,1.375222,1.0,2.602246,1.375222,2.254,1.301
1,0.0,-60.0,170.937,2,2.756588,1.950930,1.0,2.756588,1.950930,1.378,2.387
2,0.0,-60.0,6.976,3,2.609569,1.398164,1.0,2.609569,1.398164,0.000,2.610
3,0.0,-60.0,-31.365,4,2.566597,1.268904,1.0,2.566597,1.268904,-1.283,2.223
4,0.0,-60.0,55.527,5,2.658514,1.561845,1.0,2.658514,1.561845,-2.302,1.329


<h3>Set X, Y, and Z Variables</h3>
Just like in the Creating a 3D Music Model notebook, we need to get our X, Y, and Z values into 2D arrays.

In [7]:
# Store the number of segments in the song for our reshapes
num_segments = len(timbre_df.start_time.unique())

X = timbre_df.timbre_X.values.reshape(num_segments, 13).T

Y = timbre_df.timbre_Y.values.reshape(num_segments, 13).T

Z = timbre_df.start_time.values.reshape(num_segments, 13).T

<h3>Create the STL file</h3>
This boils down to only 3 lines of code. However, this was the most challenging part of the project by far. At some point I honestly believed there was no way to convert data in a <b>pandas</b> dataframe and create a 3D model with it. There is very literature, if any, on how to do this. While I was able to successfully create this one, I cannot say with completely certainty that would work for any other type of model. For this one though, what it finally came down to was having the right data in right format, and a deep understanding of the <a href = 'https://pyvista.org/'><b>pyvista</b></a> package. 

In [8]:
import pyvista as pv

# Create Structured Grid
mesh = pv.StructuredGrid(X, Y, Z)

# Convert to PolyData
poly_data = mesh.extract_surface()

# Extract to .stl
poly_data.save('song_polydata.stl')